Decision tree can be treated as a conditional probability distribution on the feature space. And it is a set of if-then rule. 

将决策树转换成if-then规则的过程如下：

* 由决策树的根节点到叶节点的每一条路径构建一条规则；
* 路径内部结点的特征对应规则的条件；(if)
* 叶节点的类对应规则的结论.(then)

且决策树的路径具有一个重要的性质：**互斥（mutually exclusive）且完备**,即每一个样本均被且只能被一条路径所覆盖。

决策树学习算法主要由三部分构成：

* 特征选择
* 决策树生成
* 决策树的剪枝

### 1. Feature selection

#### Gini impurity
Used by the CART (classification and regression tree) algorithm for classification trees, Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. 

#### Information gain

Use entropy to evaluate the feature importance. 

Let $X$ be a discrete random variable with limited values, the distribution is:

$P(X = x_i) = p_i , i = 1, 2, ..., n$

And the entropy of $X$ is 

$H(X)= - \sum_{i=1}^{n} p_{i}log(p_i)$.

(The **intuition** that we have log(p) in entrop: It's binary search, the entropy is the expection of question we need for understanding the random result. [link](https://www.youtube.com/watch?v=9r7FIXEAGvs&list=PLs8w1Cdi-zvbEpRC8YqZcSIKipAe1QHGt))

If X is 0, 1 r.v., if $P(X = 1) = p$, the entropy is:

$H(X) = - p_{i}\log(p_i) - (1 - p_{i})\log(p_i)$

Then we take the derivative: 

$\frac{\partial H(X)}{\partial p} = -log(\frac{p}{1-p})$

![](9911d23ae3bcc854e59a59365b5365be_hd.png)

So the information we gain after we know the feature could be:
$g(D,A)=H(D)-H(D|A)$ 

and $H(Y|X)$ is called mutual information. So for the train set $D$, we should find the feature $A$ that give the greatest information gain $g(D, A)$.

#### Compute the information gain (reduction in the randomness)
$g(D,A)=H(D)-H(D|A)$ 

For feature $A$, we have $a_1, a_2, ..., a_n$ different value, we can divide $D$ to $D_1, D_2, ..., D_n$ by A.

$H(D) = - \sum_{k=1}^{K}\frac{|C_k|}{|D|}log\frac{|C_k|}{|D|}$

Mutual information is just the weighted mean of entropy in $D_1, D_2, ..., D_n$.

(**Example: if A is just the class label(most informative), then Each $D_i$ has just zero entropy(well classfied.)**)

$H(D|A) = \sum_{i=1}^{n}\frac{|D_i|}{|D|}H(D_i) = \sum_{i=1}^{n}\frac{|D_i|}
{|D|}\sum_{k=1}^{K}\frac{|D_{ik}|}{D_i}log\frac{|D_{ik}|}{D_i}$

In [6]:
my_data=[['slashdot','USA','yes',18,'None'],
        ['google','France','yes',23,'Premium'],
        ['digg','USA','yes',24,'Basic'],
        ['kiwitobes','France','yes',23,'Basic'],
        ['google','UK','no',21,'Premium'],
        ['(direct)','New Zealand','no',12,'None'],
        ['(direct)','UK','no',21,'Basic'],
        ['google','USA','no',24,'Premium'],
        ['slashdot','France','yes',19,'None'],
        ['digg','USA','no',18,'None'],
        ['google','UK','no',18,'None'],
        ['kiwitobes','UK','no',19,'None'],
        ['digg','New Zealand','yes',12,'Basic'],
        ['slashdot','UK','no',21,'None'],
        ['google','UK','yes',18,'Basic'],
        ['kiwitobes','France','yes',19,'Basic']]
entropy(my_data)

1.5052408149441479

In [5]:
# 计算各个y的数量
def uniquecounts(rows):
    results = {}
    for row in rows:
        # we put the count in the last row
        r = row[-1]
        if r not in results:
            results[r] = 0
        results[r] += 1
    return results

def entropy(rows):
    from math import log
    log2 = lambda x: log(x)/log(2)
    results = uniquecounts(rows)
    ent = 0
    for r in results.keys():
        p = float(results[r])/len(rows)
        ent -= p * log2(p)
    return ent

1. ID3算法
    * select the feature with largest information gain, say A.
    * assign A as a decision attribute for Node.
    * for each value of A, create a desecendent of Node
    * sort train examples to leaves
    * recursion until the examples are well classfied.
2. C4.5
    * use gain ratio rather than information gain.
    * $ratio(D, A) = \frac{g(D, A)}{H_A(D)} $
        * where $H_A(D) = -\sum^n_{i=1}\frac{|D_i|}{D}\log_2\frac{|D_i|}{D}$ (n is the feature #values)
    * **we prefer the feature with less valus**
3. CART
    * binary tree
    * gini impurity,假设有K个类别，每个类别的概率为$p_k$. (biggest if p_k is even and K is big.)
        * $Gini(p)=\sum_{k=1}^{K}p_{k}(1-p_k)=1-\sum_{k=1}^{K}p_{k}^{2}$
        
        ![](a5f21f511372fea308ab5a2877958e77_hd.jpg)

In [ ]:
#定义节点的属性
class decisionnode:
    def __init__(self,col = -1,value = None, results = None, tb = None,fb = None):
        self.col = col   # col是待检验的判断条件所对应的列索引值
        self.value = value # value对应于为了使结果为True，当前列必须匹配的值
        self.results = results #保存的是针对当前分支的结果，它是一个字典
        self.tb = tb ## desision node,对应于结果为true时，树上相对于当前节点的子树上的节点
        self.fb = fb ## desision node,对应于结果为false时，树上相对于当前节点的子树上的节点



# 基尼不纯度


# 随机放置的数据项出现于错误分类中的概率
def giniimpurity(rows):
    total = len(rows)
    counts = uniquecounts(rows)
    imp =0
    for k1 in counts:
        p1 = float(counts[k1])/total
        for k2 in counts: # 这个循环是否可以用（1-p1）替换？
            if k1 == k2: continue
            p2 = float(counts[k2])/total
            imp+=p1*p2
    return imp


# 改进giniimpurity


def giniimpurity_2(rows):
    total = len(rows)
    counts = uniquecounts(rows)
    imp = 0
    for k1 in counts.keys():
        p1 = float(counts[k1])/total
        imp+= p1*(1-p1)
    return imp



#在某一列上对数据集进行拆分。可应用于数值型或因子型变量
def divideset(rows,column,value):
    #定义一个函数，判断当前数据行属于第一组还是第二组
    split_function = None
    if isinstance(value,int) or isinstance(value,float):
        split_function = lambda row:row[column] >= value
    else:
        split_function = lambda row:row[column]==value
    # 将数据集拆分成两个集合，并返回
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    return(set1,set2)


# 以递归方式构造树

def buildtree(rows,scoref = entropy):
    if len(rows)==0 : return decisionnode()
    current_score = scoref(rows)
    
    # 定义一些变量以记录最佳拆分条件
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    
    column_count = len(rows[0]) - 1
    for col in range(0,column_count):
        #在当前列中生成一个由不同值构成的序列
        column_values = {}
        for row in rows:
            column_values[row[col]] = 1 # 初始化
        #根据这一列中的每个值，尝试对数据集进行拆分
        for value in column_values.keys():
            (set1,set2) = divideset(rows,col,value)
            
            # 信息增益
            p = float(len(set1))/len(rows)
            gain = current_score - p*scoref(set1) - (1-p)*scoref(set2)
            if gain>best_gain and len(set1)>0 and len(set2)>0:
                best_gain = gain
                best_criteria = (col,value)
                best_sets = (set1,set2)
                
    #创建子分支
    if best_gain>0:
        trueBranch = buildtree(best_sets[0])  #递归调用
        falseBranch = buildtree(best_sets[1])
        return decisionnode(col = best_criteria[0],value = best_criteria[1],
                            tb = trueBranch,fb = falseBranch)
    else:
        return decisionnode(results = uniquecounts(rows))

# 决策树的显示
def printtree(tree,indent = ''):
    # 是否是叶节点
    if tree.results!=None:
        print str(tree.results)
    else:
        # 打印判断条件
        print str(tree.col)+":"+str(tree.value)+"? "
        #打印分支
        print indent+"T->",
        printtree(tree.tb,indent+" ")
        print indent+"F->",
        printtree(tree.fb,indent+" ")


# 对新的观测数据进行分类


def classify(observation,tree):
    if tree.results!= None:
        return tree.results
    else:
        v = observation[tree.col]
        branch = None
        if isinstance(v,int) or isinstance(v,float):
            if v>= tree.value: branch = tree.tb
            else: branch = tree.fb
        else:
            if v==tree.value : branch = tree.tb
            else: branch = tree.fb
        return classify(observation,branch)